## REST API

In [1]:
import requests
import time
import json
import os
import boto3

In [31]:
# TODO
base_url = "http://0.0.0.0:8001/api/2.0/"
username = ""
password = ""

### Create Experiment 

In [4]:
payload = {'name':'rest-demo'}

In [5]:
response = requests.post(base_url + "mlflow/experiments/create", data=json.dumps(payload), auth=(username, password))

In [6]:
print(response.json())

{'experiment_id': '5'}


### Create Model

In [19]:
payload = {'name':'matching algorithm',
          'description':'match drivers to tasks'}

In [20]:
response = requests.post(base_url + "preview/mlflow/registered-models/create", data=json.dumps(payload), auth=(username, password))

In [21]:
print(response.json())

{'registered_model': {'name': 'matching algorithm', 'creation_timestamp': '1617652051375', 'last_updated_timestamp': '1617652051375', 'description': 'match drivers to tasks'}}


### Create New Run

In [7]:
seconds = int(time.time()*1000)
print(seconds)

1617648987433


In [8]:
payload = {"name": "rest-demo",
"experiment_id":"5",
"start_time":seconds}

In [9]:
response = requests.post(base_url + "mlflow/runs/create", data=json.dumps(payload), auth=(username, password))

In [10]:
print(response.json())

{'run': {'info': {'run_uuid': 'c8c322ee8e904e64a13b4bb7174e671b', 'experiment_id': '5', 'user_id': '', 'status': 'RUNNING', 'start_time': '1617648987433', 'artifact_uri': 's3://mlflow-csco-test/5/c8c322ee8e904e64a13b4bb7174e671b/artifacts', 'lifecycle_stage': 'active', 'run_id': 'c8c322ee8e904e64a13b4bb7174e671b'}, 'data': {}}}


### Log Run Parameters

In [12]:
payload = {"run_id": "c8c322ee8e904e64a13b4bb7174e671b",
          "key":"parameter1",
          "value":"parameter1_value"}

In [13]:
response = requests.post(base_url + "mlflow/runs/log-parameter", data=json.dumps(payload), auth=(username, password))

In [15]:
print(response.json())

{}


### Log Run Metric

In [23]:
payload = {"run_id": "c8c322ee8e904e64a13b4bb7174e671b",
          "key":"metric1",
          "value":"100"} # metric value must be numeric

In [21]:
response = requests.post(base_url + "mlflow/runs/log-metric", data=json.dumps(payload), auth=(username, password))

In [22]:
print(response.json())

{}


### Log Artifact

In [32]:
# TODO
os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_SESSION_TOKEN']=''

In [3]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [17]:
upload_file('requirements.txt', 'mlflow-csco-test', '5/c8c322ee8e904e64a13b4bb7174e671b/artifacts/requirements.txt')

True

### Create Model Version

In [23]:
payload = {"name":"matching algorithm",
           "source":"s3://mlflow-csco-test/5/c8c322ee8e904e64a13b4bb7174e671b/artifacts/requirements.txt",
           "run_id": "c8c322ee8e904e64a13b4bb7174e671b",
           "run_link": "http://0.0.0.0:8001/#/experiments/5/runs/c8c322ee8e904e64a13b4bb7174e671b"}

In [24]:
response = requests.post(base_url + "preview/mlflow/model-versions/create", data=json.dumps(payload), auth=(username, password))

In [25]:
print(response.json())

{'model_version': {'name': 'matching algorithm', 'version': '1', 'creation_timestamp': '1617652209042', 'last_updated_timestamp': '1617652209042', 'current_stage': 'None', 'description': '', 'source': 's3://mlflow-csco-test/5/c8c322ee8e904e64a13b4bb7174e671b/artifacts/requirements.txt', 'run_id': 'c8c322ee8e904e64a13b4bb7174e671b', 'status': 'READY', 'run_link': 'http://0.0.0.0:8001/#/experiments/5/runs/c8c322ee8e904e64a13b4bb7174e671b'}}


### Update Model Stage

In [26]:
payload = {"name":"matching algorithm",
           "version":"1",
           "stage": "staging"}

In [27]:
response = requests.post(base_url + "preview/mlflow/model-versions/transition-stage", data=json.dumps(payload), auth=(username, password))

In [28]:
print(response.json())

{'model_version': {'name': 'matching algorithm', 'version': '1', 'creation_timestamp': '1617652209042', 'last_updated_timestamp': '1617652289197', 'current_stage': 'Staging', 'description': '', 'source': 's3://mlflow-csco-test/5/c8c322ee8e904e64a13b4bb7174e671b/artifacts/requirements.txt', 'run_id': 'c8c322ee8e904e64a13b4bb7174e671b', 'status': 'READY', 'run_link': 'http://0.0.0.0:8001/#/experiments/5/runs/c8c322ee8e904e64a13b4bb7174e671b'}}


### Update/End Run

In [7]:
seconds = int(time.time()*1000)
#print(seconds)
payload = {"run_id": "c8c322ee8e904e64a13b4bb7174e671b",
          "status":"FINISHED",
          "end_time":str(seconds)} 

In [10]:
response = requests.post(base_url + "mlflow/runs/update", data=json.dumps(payload), auth=(username, password))

In [11]:
print(response.json())

{'run_info': {'run_uuid': 'c8c322ee8e904e64a13b4bb7174e671b', 'experiment_id': '5', 'user_id': '', 'status': 'FINISHED', 'start_time': '1617648987433', 'end_time': '1617651700402', 'artifact_uri': 's3://mlflow-csco-test/5/c8c322ee8e904e64a13b4bb7174e671b/artifacts', 'lifecycle_stage': 'active', 'run_id': 'c8c322ee8e904e64a13b4bb7174e671b'}}


## Python

In [12]:
import mlflow
import os

In [29]:
# TODO
os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_SESSION_TOKEN']=''

In [30]:
# TODO
os.environ['MLFLOW_TRACKING_USERNAME']=''
os.environ['MLFLOW_TRACKING_PASSWORD']=''

In [15]:
mlflow.set_tracking_uri('http://0.0.0.0:8001')
mlflow.set_experiment('replate-demo')

In [16]:
with mlflow.start_run():
    mlflow.log_artifact('requirements.txt')